# **Model and Evaluation Trial and Errors**

## Objectives

* Establish the best fits for the fulfilfement of the Business Requirements and Hypothesis Validation.

## Inputs

* inputs/cherryleaves_dataset/cherry-leaves/test
* inputs/cherryleaves_dataset/cherry-leaves/train
* inputs/cherryleaves_dataset/cherry-leaves/validation
* image shape embeddings.

```plaintext
├── inputs
│ └── cherryleaves_dataset
│ └── cherry-leaves
│   ├── test
│   │ ├── healthy
│   │ └── powdery_mildew
│   ├── train
│   │ ├── healthy
│   │ └── powdery_mildew
│   └── validation
│   │ ├── healthy
│   │ └── powdery_mildew
└── ...
``` 

## Outputs

* Image distribution plot in train, validation, and test set
* Image augementation
* Class indices to change prediction inference in labels
* Creation of ML model
* Train ML model
* Save ML model
* Create Learning Curve Plot for model performance
* Model Evaluation on pickle file
* Prediction on the random image file

## Additional Comments

* Uncomment code which is relevant for the required tests.

---

# Import Packages

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
import tensorflow as tf
from matplotlib.image import imread

---

# Set Working Directory

In [ ]:
cwd = os.getcwd()

In [ ]:
os.chdir('/workspace/mildew-detector')
print("You set a new current directory")

In [ ]:
work_dir = os.getcwd()
work_dir

---

## Set Input Directories
Set train, validation and test paths

In [ ]:
my_data_dir = 'inputs/cherryleaves_dataset/cherry-leaves'
train_path = my_data_dir + '/train' 
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'

## Set Output Directory

In [ ]:
version = 'alfa'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

## Set Label Names

In [ ]:
labels = os.listdir(train_path)
print('Label for the images are', labels)

## Set Image Shape

In [ ]:
## Import saved image shape embedding
import joblib
version = 'v1'

# Set image shape for RGB - comment out if using rgb
image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")
image_shape

# ### SET NEW IMAGE SHAPE FOR GRAYSCALE - comment out if using grayscale
# image_shape = (256, 256, 3)
# image_shape
# joblib.dump(value=image_shape ,
#             filename=f"{file_path}/image_shape_g.pkl")

---

## Images Distribution 

### Count Number of Images per Set & Label

In [ ]:
import plotly.express as px
import pandas as pd

df_freq = pd.DataFrame([])
for folder in ['train', 'test', 'validation']:
    for label in labels:
        df_freq = df_freq.append(
            pd.Series(data={'Set': folder,
                            'Label': label,
                            'Count': int(len(os.listdir(my_data_dir + '/' + folder + '/' + label)))}
                      ),
            ignore_index=True
        )

        print(
            f"* {folder} - {label}: {len(os.listdir(my_data_dir+'/'+ folder + '/' + label))} images")

print("\n")

### Plotting the Data Using Seaborn and Matplotlib
sns.set_style("whitegrid")
plt.figure(figsize=(8, 5))
sns.barplot(data=df_freq, x='Set', y='Count', hue='Label')
plt.savefig(f'{file_path}/labels_distribution.png',
            bbox_inches='tight', dpi=150)
plt.show()

### Label Distribution - Bar Chart

In [ ]:
fig = px.bar(df_freq, 
            x="Set", 
            y="Count", 
            color='Label', 
            title="Cherry Leaves Dataset", 
            text_auto=True)
fig.update_layout(
    autosize=False,
    width=800, 
    height=500, 
    )
fig.show()
fig.write_image(f'outputs/{version}/number_leaves_sets.png')

### Set Distribution - Bar Chart

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=df_freq, x='Set', y='Count', estimator=sum, ci=None)
plt.title('Cherry leaves dataset distribution')
plt.xlabel('Set')
plt.ylabel('Count')
plt.savefig(f'{file_path}/sets_distribution_bar.png',
            bbox_inches='tight', dpi=150)
plt.show()

---

## Image Data Augmentation

### Import ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Initialize ImageDataGenerator

In [ ]:
augmented_image_data = ImageDataGenerator(rotation_range=20,
                                          width_shift_range=0.10,
                                          height_shift_range=0.10,
                                          shear_range=0.1,
                                          zoom_range=0.1,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          fill_mode='nearest',
                                          rescale=1./255
                                          )

### Set Batch Size

In [ ]:
batch_size = 18

### Augment Training Image Set

In [ ]:
train_set = augmented_image_data.flow_from_directory(train_path,
                                                     target_size=image_shape[:2],
                                                     color_mode='rgb',
                                                     batch_size=batch_size,
                                                     class_mode='categorical',
                                                     shuffle=True
                                                     )

train_set.class_indices

# ### Grayscale
# train_set = augmented_image_data.flow_from_directory(train_path,
#                                                      target_size=image_shape[:2],
#                                                      color_mode='grayscale',
#                                                      batch_size=batch_size,
#                                                      class_mode='categorical',
#                                                      shuffle=True
#                                                      )

# train_set.class_indices

### Plot Augmented Training Image Set

In [ ]:
%matplotlib inline
for _ in range(3):
    plt.figure(figsize=(2, 2))
    img, label = train_set.next()
    print(img.shape)  
    plt.imshow(img[0])
    plt.show()

# ### Grayscale
# for _ in range(3):
#     plt.figure(figsize=(2, 2))
#     img, label = train_set.next()
#     print(img.shape)  
#     plt.imshow(img[0], cmap='gray')
#     plt.show()

### Augment Validation Image Set

In [ ]:
### Softmax RGB
validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
                                                                        target_size=image_shape[:2],
                                                                        color_mode='rgb',
                                                                        batch_size=batch_size,
                                                                        class_mode='categorical',
                                                                        shuffle=False
                                                                        )

validation_set.class_indices

# ### Grayscale
# validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
#                                                                         target_size=image_shape[:2],
#                                                                         color_mode='grayscale',
#                                                                         batch_size=batch_size,
#                                                                         class_mode='categorical',
#                                                                         shuffle=False
#                                                                         )

# validation_set.class_indices

# ### Sigmoid
# validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
#                                                                         target_size=image_shape[:2],
#                                                                         color_mode='rgb',
#                                                                         batch_size=batch_size,
#                                                                         class_mode='binary',
#                                                                         shuffle=False
#                                                                         )

# validation_set.class_indices

### Plot Augmented Validation Image Set

In [ ]:
### RGB
for _ in range(3): # comment out if using grayscale
    plt.figure(figsize=(2, 2))
    img, label = validation_set.next()
    print(img.shape)  
    plt.imshow(img[0])
    plt.show()

# ### Grayscale
# for _ in range(3): # comment out if using RGB
#     plt.figure(figsize=(2, 2))
#     img, label = validation_set.next()
#     print(img.shape)
#     plt.imshow(img[0], cmap='gray')
#     plt.show()

### Augment Test Image Set

In [ ]:
### Softmax
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
                                                                  target_size=image_shape[:2],
                                                                  color_mode='rgb',
                                                                  batch_size=batch_size,
                                                                  class_mode='categorical',
                                                                  shuffle=False
                                                                  )

test_set.class_indices

# ### Grayscale
# test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
#                                                                   target_size=image_shape[:2],
#                                                                   color_mode='grayscale',
#                                                                   batch_size=batch_size,
#                                                                   class_mode='categorical',
#                                                                   shuffle=False
#                                                                   )

# test_set.class_indices

# ### Sigmoid
# test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
#                                                                   target_size=image_shape[:2],
#                                                                   color_mode='rgb',
#                                                                   batch_size=batch_size,
#                                                                   class_mode='binary',
#                                                                   shuffle=False
#                                                                   )

# test_set.class_indices

### Plot Augmented Test Image Set

In [ ]:
for _ in range(3):
    plt.figure(figsize=(2, 2))
    img, label = test_set.next()
    print(img.shape)  
    plt.imshow(img[0])
    plt.show()

### Grayscale
# for _ in range(3):
#     plt.figure(figsize=(2, 2))
#     img, label = test_set.next()
#     print(img.shape)
#     plt.imshow(img[0], cmap='gray')
#     plt.show()

### Save class_indices

In [ ]:
joblib.dump(value=train_set.class_indices,
            filename=f"{file_path}/class_indices_softmax.pkl")

## Sigmoid
# joblib.dump(value=train_set.class_indices,
#             filename=f"{file_path}/class_indices_sigmoid.pkl")

## Grayscale
# joblib.dump(value=train_set.class_indices,
#             filename=f"{file_path}/class_indices_grayscale.pkl")

---

## Model Creation

### Import Packages

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam, Adagrad

### Model

In [ ]:
### Softmax
def create_tf_model():
    model = Sequential()

    ### input layer
    model.add(Conv2D(32, (3, 3), input_shape=image_shape, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    ### convolutional layers
    model.add(Conv2D(32, (3, 3), activation='relu')) # to comment out if trialing with one less convolution layer
    model.add(MaxPooling2D(pool_size=(2, 2))) # to comment out if trialing with one less convolution layer
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    ### fully connected layer
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(.2))

    ### output
    model.add(Dense(2, activation='softmax')) 

    model.compile(loss='categorical_crossentropy', 
                  optimizer='adagrad', # to replace with other optimizers if trialing 
                  metrics=['accuracy'])

    return model

# ### Sigmoid
# def create_tf_model():
#     model = Sequential()

#     ### input layer
#     model.add(Conv2D(32, (3, 3), input_shape=image_shape, activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     ### convolutional layers
#     model.add(Conv2D(32, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))      
#     model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))

#     ### fully connected layer
#     model.add(Flatten())
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(.2))

#     ### output
#     model.add(Dense(1, activation='sigmoid')) 

#     model.compile(loss='binary_crossentropy', 
#                   optimizer='adagrad',
#                   metrics=['accuracy'])

#     return model

### Model Summary

In [ ]:
create_tf_model().summary()

### Callbacks

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(monitor='val_accuracy',mode='max',verbose=1, patience=2)
m_checkpoint = ModelCheckpoint(filepath='outputs/v1/powdery_mildew_model.h5', monitor='val_accuracy', mode='max', save_best_only=True)

### Fit Model for Training

In [ ]:
model = create_tf_model()
model.fit(train_set,
          batch_size=batch_size,
          epochs=32,
          steps_per_epoch=len(train_set.classes) // batch_size,
          validation_data=validation_set,
          callbacks=[early_stop, m_checkpoint], # comment out if removing early stop
          verbose=1
          )

### Save Model

In [ ]:
model.save('outputs/alfa/powdery_mildew_model_softmax_early.h5')
# model.save('outputs/alfa/powdery_mildew_model_softmax.h5') # uncomment if not using early stop
# model.save('outputs/alfa/powdery_mildew_model_sigmoid_early.h5') # uncomment if using sigmoid with early stop
# model.save('outputs/alfa/powdery_mildew_model_sigmoid.h5') # uncomment if using sigmoid without early stop


---

## Model Performance

### Training, Validation, Loss & Accuracy Over Multiple Epochs

In [ ]:
# Softmax with Early Stop
losses = pd.DataFrame(model.history.history)
plt.figure(figsize=(5, 4))
sns.set_style("whitegrid")
losses[['loss', 'val_loss']].plot(style='.-')
plt.title("Loss")
plt.savefig(f'{file_path}/model_training_losses_softmax_early.png',
            bbox_inches='tight', dpi=150)
plt.show()

print("\n")
losses[['accuracy', 'val_accuracy']].plot(style='.-')
plt.title("Accuracy")
plt.savefig(f'{file_path}/model_training_acc_softmax_early.png',
            bbox_inches='tight', dpi=150)
plt.show()

# # Softmax without Early Stop
# losses = pd.DataFrame(model.history.history)
# plt.figure(figsize=(5, 4))
# sns.set_style("whitegrid")
# losses[['loss', 'val_loss']].plot(style='.-')
# plt.title("Loss")
# plt.savefig(f'{file_path}/model_training_losses_softmax.png',
#             bbox_inches='tight', dpi=150)
# plt.show()

# print("\n")
# losses[['accuracy', 'val_accuracy']].plot(style='.-')
# plt.title("Accuracy")
# plt.savefig(f'{file_path}/model_training_acc_softmax.png',
#             bbox_inches='tight', dpi=150)
# plt.show()

# # Sigmoid with Early Stop
# losses = pd.DataFrame(model.history.history)
# plt.figure(figsize=(5, 4))
# sns.set_style("whitegrid")
# losses[['loss', 'val_loss']].plot(style='.-')
# plt.title("Loss")
# plt.savefig(f'{file_path}/model_training_losses_sigmoid_early.png',
#             bbox_inches='tight', dpi=150)
# plt.show()

# print("\n")
# losses[['accuracy', 'val_accuracy']].plot(style='.-')
# plt.title("Accuracy")
# plt.savefig(f'{file_path}/model_training_acc_sigmoid_early.png',
#             bbox_inches='tight', dpi=150)

# # Sigmoid without Early Stop
# losses = pd.DataFrame(model.history.history)
# plt.figure(figsize=(5, 4))
# sns.set_style("whitegrid")
# losses[['loss', 'val_loss']].plot(style='.-')
# plt.title("Loss")
# plt.savefig(f'{file_path}/model_training_losses_sigmoid.png',
#             bbox_inches='tight', dpi=150)
# plt.show()

# print("\n")
# losses[['accuracy', 'val_accuracy']].plot(style='.-')
# plt.title("Accuracy")
# plt.savefig(f'{file_path}/model_training_acc_sigmoid.png',
#             bbox_inches='tight', dpi=150)
# plt.show()

### Loss & Accuracy Over Training Epochs

In [ ]:
# Softmax with Early Stop
pd.DataFrame(model.history.history).plot(figsize=(8,5))
plt.savefig(f'{file_path}/model_loss_acc_softmax_early.png',
            bbox_inches='tight', dpi=150)
plt.show()

# # Softmax without Early Stop
# pd.DataFrame(model.history.history).plot(figsize=(8,5))
# plt.savefig(f'{file_path}/model_loss_acc_softmax.png',
#             bbox_inches='tight', dpi=150)
# plt.show()

# # Sigmoid with Early Stop
# pd.DataFrame(model.history.history).plot(figsize=(8,5))
# plt.savefig(f'{file_path}/model_loss_acc_sigmoid_early.png',
#             bbox_inches='tight', dpi=150)
# plt.show()

# # Softmax without Early Stop
# pd.DataFrame(model.history.history).plot(figsize=(8,5))
# plt.savefig(f'{file_path}/model_loss_acc_sigmoid.png',
#             bbox_inches='tight', dpi=150)
# plt.show()

### Training & Validation for Loss & Accuracy Over Epochs

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter( y=model.history.history['val_loss'], name="val_loss"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter( y=model.history.history['loss'], name="loss"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter( y=model.history.history['val_accuracy'], name="val accuracy"),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter( y=model.history.history['accuracy'], name="accuracy"),
    secondary_y=True,
)

fig.update_layout(
    title_text="Loss/Accuracy of LSTM Model"
)

fig.update_xaxes(title_text="Epoch")

fig.update_yaxes(title_text="<b>primary</b> Loss", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> Accuracy", secondary_y=True)

fig.update_layout(
    autosize=False,
    width=800, 
    height=500, 
    )

fig.show()
fig.write_image(f'{file_path}/model_history_softmax_early.png')
# fig.write_image(f'{file_path}/model_history_softmax.png') # uncomment to change for softmax without early stop
# fig.write_image(f'{file_path}/model_history_sigmoid_early.png') # uncomment to change for sigmoid with early stop
# fig.write_image(f'{file_path}/model_history_sigmoid.png') # uncomment to change for sigmoid without early stop

---

## Model Evaluation

### Import Packages

In [ ]:
import sklearn
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

### Load Saved Model

In [ ]:
from keras.models import load_model
model = load_model('outputs/alfa/powdery_mildew_model_softmax_early.h5')
# model = load_model('outputs/alfa/powdery_mildew_model_softmax.h5') # uncomment if not using early stop
# model = load_model('outputs/alfa/powdery_mildew_model_sigmoid_early.h5') # uncomment if using sigmoid with early stop
# model = load_model('outputs/alfa/powdery_mildew_model_sigmoid.h5') # uncomment if using sigmoid without early stop

### Accuracy

In [ ]:
evaluation = model.evaluate(test_set, batch_size=batch_size)
print("Model accuracy: {:.2f}%".format(evaluation[1] * 100))
print("Model Loss: ",evaluation[0])

### Set Accuracy Variables

In [ ]:
validation_set.reset()

x_true, y_true = next(test_set)
preds = np.argmax(model.predict(test_set), axis=1)
y_pred = np.rint(preds)
y_true = test_set.labels

### ROC Curve

In [ ]:
from sklearn.metrics import roc_curve, auc, accuracy_score, roc_auc_score

# Set the style of the plots using seaborn
sns.set_style("whitegrid")

# Compute the ROC curve
fpr, tpr, _ = roc_curve(y_true, y_pred)

# Calculate the area under the ROC curve (AUC)
roc_auc = auc(fpr, tpr)

# Create a new figure for the ROC curve plot
plt.figure(figsize=(8, 5))

# Set line width for the plot
lw = 2

# Plot the ROC curve
plt.plot(fpr, tpr, color='darkorange',
lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)

# Plot the diagonal line representing a random classifier
plt.plot([0, 1], [0, 1], color='navy', lw=lw, label="Random classifier", linestyle='--')

# Set the limits for the x and y axes
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

# Label the x and y axes
plt.xlabel('False Positive Rate (Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')

# Set the title of the plot
plt.title('Receiver Operating Characteristic')

# Add a legend to the plot
plt.legend(loc="lower right")

# Save the figure to a file with specified path and properties
plt.savefig(f'{file_path}/roccurve_softmax_early.png',
            bbox_inches='tight', dpi=150)
# plt.savefig(f'{file_path}/roccurve_softmax.png',
#             bbox_inches='tight', dpi=150) # uncomment for softmax without early stop
# plt.savefig(f'{file_path}/roccurve_sigmoid_early.png',
#             bbox_inches='tight', dpi=150) # uncomment for sigmoid with early stop
# plt.savefig(f'{file_path}/roccurve_sigmoid.png',
#             bbox_inches='tight', dpi=150) # uncomment for sigmoid without early stop
print('Area Under ROC-Curve: ', roc_auc_score(y_true, y_pred))
plt.show()

### Classification Report

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
print('Classification Report:\n----------------------\n')
print(classification_report(y_true, y_pred, target_names=labels))

### Confusion Matrix

In [ ]:
cm = confusion_matrix(y_true,y_pred)

# Retrieve the class labels from the test set
classes=list(test_set.class_indices.keys()) 
length=len(classes)

# Create a new figure for the confusion matrix plot
plt.figure(figsize=(8, 5))

# Plot the confusion matrix using a heatmap
sns.heatmap(cm, annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False)       

# Customize the x-axis and y-axis tick labels
plt.xticks(np.arange(length)+.5, classes, rotation= 0, fontsize=8)
plt.yticks(np.arange(length)+.3, classes, rotation=90, fontsize=8)

# Label the x-axis and y-axis
plt.xlabel("Predicted")
plt.ylabel("Actual")

# Set the title of the plot
plt.title("Confusion Matrix")

# Save the figure to a file with the specified path, ensuring tight bounding box and high resolution
plt.savefig(f'{file_path}/confusion_matrix_softmax_early.png',
            bbox_inches='tight', dpi=150)
# plt.savefig(f'{file_path}/confusion_matrix_softmax.png',
#             bbox_inches='tight', dpi=150) # uncomment for softmax without early stop
# plt.savefig(f'{file_path}/confusion_matrix_sigmoid_early.png',
#             bbox_inches='tight', dpi=150) # uncomment for sigmoid with early stop
# plt.savefig(f'{file_path}/confusion_matrix_sigmoid.png',
#             bbox_inches='tight', dpi=150) # uncomment for sigmoid without early stop
plt.show()

### Save Evaluation Pickle

In [ ]:
joblib.dump(value=evaluation,
            filename=f"outputs/alfa/evaluation.pkl")

---

## Predict On New Data

### Load Random Image as PIL

In [ ]:
from tensorflow.keras.preprocessing import image

pointer = 61
label = labels[1] # select 0 for 'healthy' or 1 for 'powdery_mildew'

pil_image = image.load_img(test_path + '/' + label + '/' + os.listdir(test_path+'/' + label)[pointer],
                           target_size=image_shape, color_mode='rgb')
print(f'Image shape: {pil_image.size}, Image mode: {pil_image.mode}')
pil_image

# ### Grayscale
# from tensorflow.keras.preprocessing import image

# pointer = 32
# label = labels[1]  # select healthy or powdery_mildew

# pil_image = image.load_img(test_path + '/' + label + '/' + os.listdir(test_path+'/' + label)[pointer],
#                            target_size=(256, 256, 1), color_mode='grayscale')
# print(f'Image shape: {pil_image.size}, Image mode: {pil_image.mode}')
# pil_image

### Convert Image to Array & Prepare for Prediction

In [ ]:
my_image = image.img_to_array(pil_image)
my_image = np.expand_dims(my_image, axis=0)/255
print(my_image.shape)

### Predict Class Probabilities

In [ ]:
pred_proba = model.predict(my_image)[0, 0]

target_map = {v: k for k, v in train_set.class_indices.items()}
pred_class = target_map[pred_proba < 0.5]

if pred_class == target_map[1]:
    pred_proba = 1 - pred_proba

print(f"{pred_class} {round(pred_proba*100, 2)}%")